# 多变量问题的优化和求根算法
（Algorithms for Optimization and Root Finding for Multivariate Problems）

## 优化算法（Optimizers）

### 牛顿-共轭梯度（Newton-Conjugate Gradient）

首先，回顾一下在一维空间下牛顿法的实现：

在一维空间优化问题的讲义中，使用了牛顿法来查找零点。牛顿法也可以用来作为一种优化方法。在后面的例子中，我们就在查找一阶导数的零点。

咱们先对比一下公式，避免混淆：

$$\begin{array}{|c|c|c|c|}
\hline
\text{Finding roots of } f  & \text{Geometric Interpretation} & \text{Finding Extrema of } f & \text{Geometric Interpretation} \\
\hline
x_{n+1} = x_n -\frac{f(x_n)}{f'(x_n)} &\text{Invert linear approximation to }f & x_{n+1} = x_n -\frac{f'(x_n)}{f''(x_n)}& \text{Use quadratic approximation of } f \\
\hline
\end{array}$$

这是从两种不同角度来看待同一个问题。例如，在求根问题中线性近似实际上就是优化过程中二次近似的导函数。

#### 海森矩阵（Hessian），梯度（Gradients）和各种形式（Forms） 

回顾一下对多变量函数进行优化的原理。回想一下单变量的情况，极值（extreme values）（局部极值，local extreme）取在一阶导数为零的点，然而一阶导数为零这一个条件还并不足以判定就是局部的最大值或者最小值。通常我们还要用二阶导数来衡量一下备选的点是不是最大值或者最小值的点（如果二阶导数不存在或者为零，就不是最值）。在多变量的情况下，一阶导数和二阶导数都是*矩阵*了。在$\mathbb{R}^n$上的标量值函数中，一阶导数是一个$n\times 1$向量，称为*梯度*，记作$\nabla f$。二阶导数是一个$n\times n$矩阵，称为*海森矩阵*，记作$H$。

梯度和海森矩阵的表达式如下所示：

$$\nabla f(x) = \left(\begin{matrix}\frac{\partial f}{\partial x_1}\\ \vdots \\\frac{\partial f}{\partial x_n}\end{matrix}\right)$$


$$H = \left(\begin{matrix}
  \dfrac{\partial^2 f}{\partial x_1^2} & \dfrac{\partial^2 f}{\partial x_1\,\partial x_2} & \cdots & \dfrac{\partial^2 f}{\partial x_1\,\partial x_n} \\[2.2ex]
  \dfrac{\partial^2 f}{\partial x_2\,\partial x_1} & \dfrac{\partial^2 f}{\partial x_2^2} & \cdots & \dfrac{\partial^2 f}{\partial x_2\,\partial x_n} \\[2.2ex]
  \vdots & \vdots & \ddots & \vdots \\[2.2ex]
  \dfrac{\partial^2 f}{\partial x_n\,\partial x_1} & \dfrac{\partial^2 f}{\partial x_n\,\partial x_2} & \cdots & \dfrac{\partial^2 f}{\partial x_n^2}
\end{matrix}\right)$$


关于海森矩阵有一个最重要的性质一定要注意到，就是它是对称的（symmetric）。这个结构会引出很多有用的性质，能用于推导（interpreting）关键点（critical points）。

多变量模拟证明了局部最大值或最小值取决于梯度和海森矩阵。具体来说就是一个函数$f:\mathbb{R}^n\rightarrow \mathbb{R}$，如果有$\nabla f(x) = 0$（注意这里的0 是一个零向量），则在$x$位置有一个关键点（critical point）。更进一步，用二阶导数对关键点进行测试的规则如下：

* 如果 $H(x)$ 正定（positive-definite）, $f$ 在 $x$ 有局部最小值；
* 如果 $H(x)$ 负定（negative-definite）, $f$ 在 $x$ 有局部最大值；
* 如果 $H(x)$ 的特征值（eigenvalues）有正有负, $f$ 在 $x$ 有一个马鞍点（Saddle Point）。

### 牛顿共轭梯度算法（Newton CG Algorithm）

特点（Features）:

* 用 $n$ 步对$\mathbb{R}^n$上的“真（true）”二次方进行最小化（Minimizes a 'true' quadratic on $\mathbb{R}^n$ in $n$ steps）；
* 不需要存储或者逆转一个$n \times n$矩阵（Does NOT require storage or inversion of an $n \times n$ matrix.）。

先从 $:\mathbb{R}^n\rightarrow \mathbb{R}$开始，对$f$取一个二次近似（quadratic approximation）：

$$f(x) \approx \frac12 x^T H x + b^Tx + c$$

注意在最小值的邻域，海森矩阵$H$是正定对称的(positive-definite and symmetric)。对于最大化的过程，就考虑$-H$就可以了。

这样就把原本的优化问题化简为查找下面方程零点的问题了：
$$Hx = -b$$

这是一个线性问题了，就很赞，因为好解决。这里的维度$n$可能会很大，这就不那么好了。另外，*经验上来看*似乎我们需要知道$H$。然而实际上并不需要，不过这需要额外解释一下才能明白。

#### 广义内积（General Inner Product）

回忆一下内积（inner product）的公理定义（axiomatic definition）$<,>_A$:

* 对任意的两个向量 $v,w$ 有
$$<v,w>_A = <w,v>_A$$
* 对任意的一个向量 $v$
$$<v,v>_A \;\geq 0$$
   等价于 $\iff$ $v=0$.
* 对 $c\in\mathbb{R}$ ， $u,v,w\in\mathbb{R}^n$，则有
$$<cv+w,u> = c<v,u> + <w,u>$$

这些特征分别就是对称性（symmetric），正定（positive definite）和双线性（bilinera）。

定理：如果我们把在$\mathbb{R}^n$ 上的标准内积表示为as $<,>$ (注意这里不是 '点积dot product')，任意的一个对称、正定的$n\times n$ 矩阵 $A$ 就通过如下方式定义了在$\mathbb{R}^n$上的内积：

$$<v,w>_A \; = <v,Aw> = v^TAw$$

跟标准内积（standard inner product）类似，广义内积（general inner product）定义了一种”正交关系（orthogonality）“记号。回忆一下，在标准内积的情况中如果两个向量的内积为零（vanishes），则这两个向量正交（orthogonal）。同理也适用于$<,>_A$：

$$<v,w>_A = 0 $$

这意味着$v$和$w$这两个向量在矩阵$A$引入（induced）的内积下（inner product）是正交的。也就是说，如果$v,w$ 两个向量在矩阵$A$下是正交的，则有：

$$v^TAw = 0$$

这种关系也叫做”共轭（conjugate）“，正对应的是标题中 CG 这个缩写。

#### 共轭向量（Conjugate Vectors）

假如我们有一系列的 $n$ 个向量$p_1,...,p_n$，这些向量都是相互共轭的。那么这些向量就组成了$\mathbb{R}^n$的一个基（basis）。回到眼前的话题，这就意味着上文化简得到的那个线性问题的解向量$x$，可以写成下面的形式：

$$x = \sum\limits_{i=1}^n \alpha_i p_i$$

所以，问题就从寻找$x$继续精简到查找一组共轭基向量和$x$在这个基向量上的系数（coefficients）。


因为：

$${p}_k^{T} {b}={p}_k^{T} {A}{x}$$

另外因为 $x = \sum\limits_{i=1}^n \alpha_i p_i$所以有：
$$p^TAx = \sum\limits_{i=1}^n \alpha_i p^TA p_i$$

这样就可以解 $\alpha_k$:

$$\alpha_k = \frac{{p}_k^{T}{b}}{{p}_k^{T} {A}{p}_k} = \frac{\langle {p}_k, {b}\rangle}{\,\,\,\langle {p}_k,  {p}_k\rangle_{A}} = \frac{\langle{p}_k, {b}\rangle}{\,\,\,\|{p}_k\|_{A}^2}.$$


现在我们就只需要“$p_k$”了。

初始猜测可以采取在某个初始点$x_1$的梯度。所以就设$p_1 = \nabla f(x_1)$。然后设：

$$x_2 = x_1 + \alpha_1p_1$$

看上去挺眼熟吧。实际上这就是梯度下降法（gradient descent）。对$p_2$，我们希望$p_1$和$p_2$是共轭向量（在$A$下）。这就意味着在$A$的内积下这两个向量是正交的。设：

$$p_2 = \nabla f(x_1) - \frac{p_1^TA\nabla f(x_1)}{{p}_1^{T}{A}{p}_1} {p}_1$$

例如，我们去在$x_1$位置的梯度，然后减去在$p_1$上的投影。这就和格兰姆-施密特正交化（Gram-Schmidt orthogonalization）是一样的。

第 $k^{th}$ 个共轭向量为：

$$p_{k+1} = \nabla f(x_k) - \sum\limits_{i=1}^k\frac{p_i^T A \nabla f(x_k)}{p_i^TAp_i} p_i$$

通常这里有一个“技巧”，就是我们往往并不需要全部的$n$个共轭向量。

收敛速率（Convergence rate）取决于稀疏度（sparsity）以及$A$的条件数（condition number）。最差的情况也是$n^2$。

### BFGS - Broyden–Fletcher–Goldfarb–Shanno

BFGS 是一种拟牛顿法（'quasi' Newton method）。这种方法基于牛顿法进行了改进，其中的海森矩阵$H$被替换成了近似值。

例如，假如我们要解下面这个方程：

$$B_k{p}_k = -\nabla f({x}_k)$$

关于 $p_k$。者给出了我们搜索的方向，接下来备选点（candidate point）给出如下：

$$x_{k+1} = x_k + \alpha_k p_k$$

其中参数 $\alpha_k$ 是步长。

在每一步，我们都要求新的海森矩阵估计值$H$满足割线条件（secant condition）：

$$B_{k+1}(x_{k+1}-x_k) = \nabla f(x_{k+1}) -\nabla f(x_k)$$

有一个独特的第一步的更新，能够满足上面的关系：

$$B_{k+1} = B_k + c_k v_kv_k^T$$

其中

$$ c_k = -\frac{1}{\left(B_k(x_{k+1}-x_k) - (\nabla f(x_{k+1})-\nabla f(x_k)\right)^T (x_{k+1}-x_k) }$$ 

另有

$$v_k = B_k(x_{k+1}-x_k) - (\nabla f(x_{k+1})-\nabla f(x_k))$$

***Note that the update does NOT preserve positive definiteness if $c_k<0$.  In this case, there are several options for the rank one correction, but we will not address them here.  Instead, we will describe the BFGS method, which almost always guarantees a positive-definite correction.  Specifically:***


注意，这里如果$c_k<0$，则更新不会保持正定。
这种情况下，有几种备选的选项给首步更新（rank one correction），但这里先不讲那么细致。我们先要讲一下 BFGS 方法，这个方法几乎总能保证正定的更新。

具体为：

$$B_{k+1} = B_k + b_k g_k g_k^T + c_k B_k d_k d_k^TB_k$$ 

***where we have introduced the shorthand:***

引入化简形式：

$$g_k = \nabla f(x_{k+1}) - \nabla f(x_k) \;\;\;\;\;\;\;\ \mathrm{ 和 }\;\;\;\;\;\;\; d_k = x_{k+1} - x_k$$

如果我们设

$$b_k = \frac{1}{g_k^Td_k} \;\;\;\;\; \mathrm{ 以及 } \;\;\;\;\; c_k = \frac{1}{d_k^TB_kd_k}$$

就能满足割线条件（secant condition）了。

### Nelder-Mead法（Nelder-Mead Simplex）或称下山单纯形法

牛顿法被看做是一种“二阶方法（second order method）”，因为需要二阶导数；而拟牛顿法是一阶的，只需要求一阶导数；而Nelder-Mead法（Nelder-Mead Simplex）或称下山单纯形法，则是零阶方法。也就是说，NM 法只需要函数本身，不用去求导。

对$f:\mathbb{R}^n\rightarrow \mathbb{R}$，该算法在一个$n$维度的单纯形（simplex）上计算函数值，这个单纯形是用$n+1$个顶点构造的。对于一个单变量函数，单纯形就是一个线段；在二维空间中，单纯形就是一个三角形；在三维空间中，就是一个四面体，以此类推。

这个算法从$n+1$个点开始，然后按照下面的步骤重复直到收敛：

* 计算每个点的函数值
* 整理排序这些函数值使
$$f(x_1)\leq ...\leq f(x_{n+1})$$
* 在通过$x_1,...,x_n$定义的一个$n$维度区域中计算重心（centroid） $x_c$
* 关于重心（centroid）来反射（Reflect）$x_{n+1}$来得到$x_r$
$$x_r = x_c + \alpha (x_c - x_{n+1})$$
* 创建一个新的单纯形，根据下面的规则：
    - 如果 $f(x_1)\leq f(x_r) < f(x_n)$, 将 $x_{n+1}$ 替换为 $x_r$
    - 如果 $f(x_r)<f(x_1)$, 扩展单纯性，通过 $x_r$:
    $$x_e = x_c + \gamma (x_c - x_{n+1})$$
      如果 $f(x_e)<f(x_r)$, 把 $x_{n+1}$ 替换为 $x_e$,否则将 $x_{n+1}$ 替换成 $x_r$
    - 如果 $f({x}_{r}) \geq f({x}_{n})$, 计算 $x_p = x_c + \rho(x_c - x_{n+1})$.  If $f({x}_{p}) < f({x}_{n+1})$,将 $x_{n+1}$ 替换为 $x_p$
    - 如果全部失败，就把除了$x_1$之外的所有点使用下面的公式来进行覆盖
    $$x_i = {x}_{1} + \sigma({x}_{i} - {x}_{1})$$
    
SciPy 的文档里面没有列出$\alpha, \gamma,\rho$ 和 $\sigma$这些变量的默认值，也不输入进函数。

### 波尔法（Powell's Method）

波尔法（Powell's method）是另外一种不用求导数的优化方法，有点类似共轭梯度法（conjugate-gradient）。这个算法的步骤如下所示：

从某个起始点 $p_0$ (一个初始猜测点) 和一个向量集 $\xi_1,...,\xi_n$作为$\mathbb{R}^n$的初始标准基开始。

- 计算 对于 $i=1,...,n$, 查找 $\lambda_i$ 使最小化 $f(p_{i-1} +\lambda_i \xi_i)$ 然后设 $p_i = p_{i-1} + \lambda_i\xi_i$
- 对于 $i=1,...,n-1$, 将 $\xi_{i}$ 替换成 $\xi_{i+1}$ 然后将 $\xi_n$ 替换成 $p_n - p_0$
- 选取 $\lambda$ 使 $f(p_0 + \lambda(p_n-p_0)$ 最小，然后将$p_0$ 替换成 $p_0 + \lambda(p_n-p_0)$

这个算法的核心思想是进行线性搜索，试图寻找有效的搜索方向。

## 求解方法（Solvers）

### 非线性回归法（Levenberg-Marquardt，也称阻尼最小二乘法Damped Least Squares)

回忆一下最小二乘法问题：

给定一个点集$(x_i, y_i)$，其中$x_i$是定义在$\mathbb{R}^n$上的独立变量，$y_i$是定义在$\mathbb{R}^n$上的响应变量，找出模型$f(x;\beta)$的参数值$\beta$，使之满足：

$$S(\beta) = \sum\limits_{i=1}^m \left(y_i - f(x_i;\beta)\right)^2$$
最小。

如果我们用牛顿法，更新规则如下所示：

$$\beta_{k+1} = \beta_k - H^{-1}\nabla S(\beta_k)$$

梯度下降法，则如下所示：

$$\beta_{k+1} = \beta_k - \gamma\nabla S(\beta_k)$$

非线性回归法（Levenberg-Marquardt）在牛顿法和梯度下降法之间自适应切换（adaptively switches）：

$$\beta_{k+1} = \beta_k - (H + \lambda I)^{-1}\nabla S(\beta_k)$$

如果$\lambda$很小，那么更新就主要使用牛顿-高斯法（Newton-Gauss），如果$\lambda$很大，就使用梯度下降法（gradient descent）。

### 牛顿 - 克雷洛夫法（Newton-Krylov）

克雷洛夫空间（Krylov space）的概念（notion）来自凯利-汉密尔顿定理（Cayley-Hamilton theorem，缩写为CH）。CH 表明了一个矩阵$A$满足其特征多项式（characteristic polynomial）一个直接的推论（corollary）是$A^{-1}$ 可以写成一个矩阵的权（powers）的线性组合（linear combination），其中最高的权（power）为$n-1$。

由一个$n\times n$矩阵$A$和一个$n$维度向量$b$生成的$r$阶克雷洛夫空间如下所示：

$$\mathcal{K}_r(A,b) = \operatorname{span} \, \{ b, Ab, A^2b, \ldots, A^{r-1}b \}$$

这些实际上是我们在牛顿共轭法（Newton-CG）中提到的共轭向量（conjugate vectors）所张成的子空间（subspaces spanned），所以可以说牛顿共轭法就是一种克雷洛夫方法（Krylov method）。

目前在 scipy.optimize 包中，牛顿-克雷洛夫求解发的名字为'Jacobian Free Newton Krylov'。在解*大规模*的$n\times n$非线性方程组的时候，这种算法很有效。具体细节就不说那么多了，也不给大家展示这个算法的步骤了，不过这个方法在物理学领域以及一些非线性动力学（non-linear dynamics）问题中会用到的更多一些。

## 通用线性模型估计（GLM Estimation）和迭代重新加权的最小二乘法（iteratively reweighted least squares，IRLS）

回忆一下广义线性模型，有以下要素构成：

* 线性预测器（linear predictor） $\eta = X\beta$
* 在指数族（exponential family）中分布的一个响应变量（A response variable）
* 一个可逆的（invertible）'链接（link）'函数$g$满足
$$E(Y) = \mu = g^{-1}(\eta)$$

对数似然率（log-likelihood）如下所示：

$$\ell(\eta) = \sum\limits_{i=1}^m (y_i \log(\eta_i) + (\eta_i - y_i)\log(1-eta_i) $$

其中 $\eta_i = \eta(x_i,\beta)$.

对其微分（differentiating），则得到了：

$$\frac{\partial L}{\partial \beta} = \frac{\partial \eta}{\partial \beta}^T\frac{\partial L}{\partial \eta} = 0$$

跟之前章节里面的形式有点不同，$\beta$的牛顿更新规则为：

$$-\frac{\partial^2 L}{\partial \beta \beta^T} \left(\beta_{k+1} -\beta_k\right) = \frac{\partial \eta}{\partial \beta}^T\frac{\partial L}{\partial \eta}$$

然后如果我们计算：

$$-\frac{\partial^2 L}{\partial \beta \beta^T} = \sum \frac{\partial L}{\partial \eta_i}\frac{\partial^2 \eta_i}{\partial \beta \beta^T} - \frac{\partial \eta}{\partial \beta}^T \frac{\partial^2 L}{\partial \eta \eta^T}  \frac{\partial \eta}{\partial \beta}$$

把期望值（expected values）放到等号右侧然后得到：

$$E\left(\frac{\partial L}{\partial \eta_i} \right) = 0$$

然后

$$E\left(-\frac{\partial^2 L}{\partial \eta \eta^T} \right) = E\left(\frac{\partial L}{\partial \eta}\frac{\partial L}{\partial \eta}^T\right) \equiv A$$

So if we replace the Hessian in Newton's method with its expected value, we obtain:

所以只要我们把牛顿法当中的海森矩阵（Hessian）替换成期望值（expected value），就得到了：

$$\frac{\partial \eta}{\partial \beta}^TA\frac{\partial \eta}{\partial \beta}\left(\beta_{k+1} -\beta_k\right) = \frac{\partial \eta}{\partial \beta}^T\frac{\partial L}{\partial \eta} $$

现在，实际上就已经有了加权最小二乘法问题的正则方程（normal equations for a weighted least squares problem）的形式了：

$$\min_{\beta_{k+1}}\left(A^{-1}\frac{\partial L}{\partial \eta} + \frac{\partial \eta}{\partial \beta}\left(\beta_{k+1} -\beta_k\right)\right)^T A \left(A^{-1}\frac{\partial L}{\partial \eta} + \frac{\partial \eta}{\partial \beta}\left(\beta_{k+1} -\beta_k\right)\right)$$

$A$是权值矩阵（weight matrix），随着迭代过程而改变，所以这个方法就是*迭代重新加权最小二乘法（iteratively reweighted least squares）*。

### 约束优化（Constrained Optimization）和拉格朗日乘数（Lagrange） Multipliers

通常情况下，我们在一个或者多个约束条件下对一个函数进行优化。最常见的应对这种情况的技巧就是”拉格朗日乘数法“。这个理论如下所述：

如果有一个函数$f(x,y)$，我们要对其进行优化，而约束条件为$g(x,y)=c$，那么实际上我们要找的是$f$和$g$这两个函数有同一方向梯度的点。也就相当于：

$$\nabla_{(x,y)}f = \lambda \nabla_{(x,y)}g$$

通常情况下，会在$\lambda$前面写个负号。上面这个二维问题中，有两个方程，而有三个未知数，而原始的约束条件$g(xy,)=c$构成了第三个方程。更多的附加约束可以用下面的方法来处理：

$$\nabla_{(x,y)}f = \lambda_1 \nabla_{(x,y)}g_1 + ... + \lambda_k \nabla_{(x,y)}g_k$$

这里还可以泛化上面的函数，从二维到定义在$\mathbb{R}^n$整个实数域上面：

$$\nabla_{x}f = \lambda \nabla_{x}g$$